In [ ]:
import boto3, os

In [ ]:
ec2 = boto3.resource('ec2')
ssm = boto3.client('ssm')
route53 = boto3.client('route53')

In [ ]:
default_ip_security_group_name = 'default-using-my-ip'
security_group_name = 'master-class-security-group'
security_group_description = 'security group for ec2 masterclass'

ssh_directory_name = '/Users/dougjam/.ssh'
key_pair_name = 'master-class-key-pair'
private_key_file_name = f'{ssh_directory_name}/{key_pair_name}_rsa'
public_key_file_name = f'{private_key_file_name}.pub'
ssh_config_file_name = f'{ssh_directory_name}/config'
ssh_known_hosts_file_name = f'{ssh_directory_name}/known_hosts'

domain_name = 'doug-nicholson.net'
subdomain_name = 'master-class'
full_host_name = f'{subdomain_name}.{domain_name}'

parameter_path = '/aws/service/ami-amazon-linux-latest/'
parameter_name = 'amzn2-ami-hvm-x86_64-gp2'

In [ ]:
# In the interests of security, there is a security group with
# my ip address in it.  Get the ip address range out of it for
# creating the security group the instance will use.
ip_ranges = [{'CidrIp': '0.0.0.0/0'}]
security_group_iterator = ec2.security_groups.filter(
    Filters=[
        {
            'Name': 'group-name',
            'Values': [
                default_ip_security_group_name
            ]
        }
    ]
)
for security_group in security_group_iterator:
    for ip_permission in security_group.ip_permissions:
        ip_ranges = ip_permission['IpRanges']

In [ ]:
# Create a new security group for the class.
security_group = ec2.create_security_group(
    GroupName=security_group_name,
    Description=security_group_description
)
# Add the ingress rules to it.
response = security_group.authorize_ingress(
    IpPermissions=[
        {'FromPort': 22,
        'ToPort': 22,
        'IpProtocol': 'tcp',
        'IpRanges': ip_ranges
        }
    ]
)

In [ ]:
# Remove the host block for the class from the ssh
# config file if it is in there.
ssh_config_lines = []
if os.path.isfile(ssh_config_file_name):
    with open(ssh_config_file_name, 'r') as f:
        ssh_config_lines = f.readlines();
# Add the host block for the class
ssh_config_lines.append('\n')
ssh_config_lines.append(f'Host {full_host_name}\n')
ssh_config_lines.append(f'    IdentityFile ~/.ssh/{key_pair_name}_rsa\n')
# Write the lines back to the file.
with open(ssh_config_file_name, 'w') as f:
    f.writelines(ssh_config_lines)

In [ ]:
# Use ssh-keygen to create the new key files.
os.system(f'ssh-keygen -q -N "" -f {private_key_file_name}')
os.chmod(private_key_file_name, 0o600)
os.chmod(public_key_file_name, 0o600)

In [ ]:
# Import the key pair.
with open(public_key_file_name, 'r') as f:
    public_key_material = f.read()
response = ec2.import_key_pair(
    KeyName=key_pair_name,
    PublicKeyMaterial=public_key_material
)

In [ ]:
response = ssm.get_parameters_by_path(
    Path=parameter_path)
for parameter in response['Parameters']:
    if parameter['Name'] == parameter_path + parameter_name:
        image_id = parameter['Value']
        break

In [ ]:
# create the basic instance
instances = ec2.create_instances(
    ImageId=image_id,
    InstanceType='t2.micro',
    KeyName=key_pair_name,
    MaxCount=1,
    MinCount=1,
    SecurityGroups=[
        security_group_name
    ]
)
for instance in instances:
    instance.wait_until_running()
    instance.reload()

In [ ]:
# add the public ip address to route53 so SSH is easier
route53_waiter = route53.get_waiter('resource_record_sets_changed')
response = route53.list_hosted_zones_by_name(
    DNSName=domain_name
)
zone_id = response['HostedZones'][0]['Id'][12:]
response = route53.change_resource_record_sets(
    HostedZoneId=zone_id,
    ChangeBatch={
        'Changes': [
            {
                'Action': 'UPSERT',
                'ResourceRecordSet': {
                    'Name': full_host_name,
                    'Type': 'A',
                    'TTL': 60,
                    'ResourceRecords': [
                        {
                            'Value': instance.public_ip_address
                        },
                    ],
                }
            }
        ]
    }
)
route53_waiter.wait(
    Id=response['ChangeInfo']['Id'][8:]
)

In [ ]:
print('new instance running')